In [ ]:
# Importação de bibliotecas externas necessárias
from deap import base, creator, tools, benchmarks
import deap.cma as cma

import copy
import random
import math
import time
import logging
import threading

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

from scipy.integrate import odeint
from scipy.integrate import solve_ivp

import sys
import os

# Importação de módulos personalizados
sys.path.append(os.path.abspath("..")) 
from Modules.Helper import Helper
from Modules.Solvers import Solvers
from Modules.Plotters import Plotters
from Modules.Equation import Equation

import queue
from concurrent.futures import ThreadPoolExecutor, as_completed

In [2]:
# Carrega os dados do sistema GRN5 e define as condições iniciais
labels = ['A', 'B', 'C', 'D', 'E']
df, max_data = Helper.load_data(filename='../Data/GRN5_DATA.txt', labels=labels)
initial_conditions = np.array([df[label].iloc[0] for label in labels])
t_span = (df['t'].iloc[0], df['t'].iloc[-1])  # Intervalo de tempo para simulações
t_eval = np.array(df['t'])  # Ponto de avaliação dos dados temporais
original = np.array(df[labels]).T  # Dados originais para cálculo de erro

# Limites nos valores dos coeficientes tau, k e n 
bounds = {
    'tau': (0.1, 5.0),
    'k': (0.1, 2.0),
    'n': (0.1, 30.0)
}

IND_SIZE = 19  # Tamanho do indivíduo (número total de coeficientes)
GENERATION_LIMIT = 5000
NO_IMPROVEMENT_LIMIT = 50
HARD_SIGMA_INCREASE_FACTOR = 10
TOLERANCE = 1E-4

In [3]:
# Representa um coeficiente com valor e limites
class Coefficient:
    def __init__(self, bounds_val):
        self.val = random.uniform(*bounds_val)  # Inicializa com valor aleatório dentro dos limites
        self.bounds_val = bounds_val
    
    def __repr__(self):
        return f"val={self.val}"

# Representa um coeficiente usado no CMA-ES com limites.
class CMACoefficient:
    def __init__(self, val, bounds_val):
        self.bounds_val = bounds_val
        self.val = self.limit_val(val)  # Ajusta o valor aos limites
    
    # Garante que o valor esteja dentro dos limites
    def limit_val(self, val):
        return max(self.bounds_val[0], min(val, self.bounds_val[1]))
    
    def __repr__(self):
        return f"val={self.val}"

# Representa um indivíduo contendo coeficientes e funções para manipulação
class Individual:
    def __init__(self, method='RK45', error='ABS'):
        # Estrutura do indivíduo contendo os coeficientes para cada variável
        self.coeffs = {
            'A': {
                'E': {'n': None, 'k': None, '-': True},
                'tau': None
            },
            'B': {
                'A': {'n': None, 'k': None, '-': False},
                'tau': None
            },
            'C': {
                'B': {'n': None, 'k': None, '-': False},
                'tau': None,
            },
            'D': {
                'C': {'n': None, 'k': None, '-': False},
                'tau': None,
            },
            'E': {
                'D': {'n': None, 'k': None},
                'B': {'n': None, 'k': None},
                'E': {'n': None, 'k': None},
                'tau': None,
            }
        }
        
        # Tamanho do indivíduo
        self.ind_size = IND_SIZE 
        
        # Fitness inicializado como infinito
        self.fitness = np.inf
        self.method = method
        self.error = error
     
    @staticmethod
    def list_to_ind(list_ind, method, error):
        i = 0
        ind = Individual(method=method, error=error)
        for key, label in ind.coeffs.items():
            label['tau'] = CMACoefficient(list_ind[i], bounds['tau'])
            i += 1
            for key, coeffs in label.items():
                if key != 'tau':
                    coeffs['n'] = CMACoefficient(list_ind[i], bounds['n'])
                    coeffs['k'] = CMACoefficient(list_ind[i+1], bounds['k'])
                    i += 2
        return ind
    
    def ind_to_list(self):
        ind_list = []
        for key, label in self.coeffs.items():
            ind_list.append(label['tau'].val)
            for key, coeffs in label.items():
                if key != 'tau':
                    ind_list.append(coeffs['n'].val)
                    ind_list.append(coeffs['k'].val)
        return ind_list
    
    @staticmethod
    def apply_bounds(population, method, error):
        for ind in population:
            list_ind = Individual.list_to_ind(ind, method, error)
            ind[:] = Individual.ind_to_list(list_ind)
    
    @staticmethod    
    def cma_evaluate(method, error, list_ind):
        ind = Individual.list_to_ind(list_ind, method, error)
        ind.calc_fitness()
        return ind.fitness,
    
    # Calcula o fitness do indivíduo simulando o sistema de ODEs
    def calc_fitness(self):
        try:
            equation = Equation(self.numerical_coeffs, labels)
            # Métodos do solve_ivp: RK45, DOP853, LSODA, BDF... 
            y = solve_ivp(self.system, t_span, initial_conditions, method=self.method, t_eval=t_eval, args=(equation, ), min_step=0.001).y
            
            if self.error == 'ABS':
                self.fitness = self.abs_error(original, y)
            elif self.error == 'MSE':
                self.fitness = self.MSE_error(original, y)
            elif self.error == 'MABS':
                self.fitness = self.mean_abs_error(original, y)
            elif self.error == 'SQUARED':
                self.fitness = self.squared_error(original, y)
            
            self.fitness = min(self.fitness, 1e6)
        except:
            # Trata exceções relacionadas ao solver
            print("Fitness Overflow")
            self.fitness = 1e6
            
    def calc_all_fitness(self):
        equation = Equation(self.numerical_coeffs, labels)
        # Métodos do solve_ivp: RK45, DOP853, LSODA, BDF... 
        y = solve_ivp(self.system, t_span, initial_conditions, method=self.method, t_eval=t_eval, args=(equation, ), min_step=0.001).y
        
        return {
            'ABS_Fitness': self.abs_error(original, y), 
            'SQUARED_Fitness': self.squared_error(original, y),
            'MSE_Fitness': self.MSE_error(original, y), 
            'MABS_Fitness': self.mean_abs_error(original, y), 
        }
     
       
    @staticmethod
    def system(t, y, equation):
        vals = [Solvers.norm_hardcoded(val, max_data[label]) for val, label in zip(y, labels)]
        N_A, N_B, N_C, N_D, N_E = vals
        
        dA = equation.full_eq(vals, 'A', 'E')
        dB = equation.full_eq(vals, 'B', 'A')
        dC = equation.full_eq(vals, 'C', 'B')
        dD = equation.full_eq(vals, 'D', 'C')
        dE = equation.complex_eqs(vals, 'E', [['+B', '+D'], ['+D', '+E']])

        return [dA, dB, dC, dD, dE]
    
    @staticmethod
    def abs_error(original, pred):
        return sum(sum(abs(original-pred)))
    
    @staticmethod
    def squared_error(original, pred):
        return sum(sum( (original-pred)**2 ))**(1/2)
    
    @staticmethod
    def MSE_error(original, pred):
        return np.mean((original-pred)**2)
    
    @staticmethod
    def mean_abs_error(original, pred):
        return np.mean(abs(original-pred))
            
    @staticmethod
    def initialize_ind(bounds):
        ind = Individual()
        for key, label in ind.coeffs.items():
            label['tau'] = Coefficient(bounds['tau'])
            for key, coeffs in label.items():
                if key != 'tau':
                    coeffs['n'] = Coefficient(bounds['n'])
                    coeffs['k'] = Coefficient(bounds['k'])
                    
        ind.calc_fitness()
        return ind
    
    @property
    def numerical_coeffs(self,):
        
        numerical_coeffs = copy.deepcopy(self.coeffs)
        for key, label in numerical_coeffs.items():
            label['tau'] = label['tau'].val
            for key, coeffs in label.items():
                if key != 'tau':
                    coeffs['n'] = int(coeffs['n'].val)
                    coeffs['k'] = coeffs['k'].val
                    
        return numerical_coeffs
    
    def plot(self, method='RK45', comparison=True):
        methods = [self.method]
        results = {}
        equation = Equation(self.numerical_coeffs, labels)
        # Métodos do solve_ivp: RK45, DOP853, LSODA, BDF... 
        results[method] = solve_ivp(self.system, t_span, initial_conditions, method=method, t_eval=t_eval, args=(equation, ), min_step=0.001).y
        Plotters.plot_methods(results=results,t=t_eval, methods=methods, labels=labels)
        if comparison:
            Plotters.plot_comparison(results=results, t=t_eval, df=df, methods=methods, labels=labels)
            
    @staticmethod        
    def initialize_average_bounds(bounds, ind_size):
        averages = [np.mean(value) for value in bounds.values()]
        array = np.resize(averages, ind_size)
        return array   
        
    def __repr__(self):
        coeffs_repr = {k: v for k, v in self.coeffs.items()}
        return f"Individual (fitness={self.fitness}, coeffs={coeffs_repr}, ind_size={self.ind_size})"

In [4]:

creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", list, fitness=creator.FitnessMin)

def instantiate_toolbox():
    # Criação do tipo de indivíduo e toolbox
    toolbox = base.Toolbox()

    # Inicializa estratégia CMA-ES
    centroids = Individual.initialize_average_bounds(bounds, IND_SIZE)
    strategy = cma.Strategy(centroid=centroids, sigma=10, lambda_=int(4+(3*np.log(IND_SIZE))))

    toolbox.register("generate", strategy.generate, creator.Individual)
    toolbox.register("update", strategy.update)

    # Estatísticas e parâmetros do algoritmo
    hof = tools.HallOfFame(1)
    stats = tools.Statistics(lambda individual: individual.fitness.values)
    stats.register("avg", np.mean)
    stats.register("std", np.std)
    stats.register("min", np.min)
    stats.register("max", np.max)
    return toolbox, hof, strategy, stats

In [5]:
def run_evolution(seed, error, method, result_container, timeout_limit):
    logging.info(f"Starting execution for: Solver={method}, Error={error}, Seed={seed}")
    print(f"Solver: {method}, Error: {error}")

    toolbox, hof, strategy, stats = instantiate_toolbox()
    np.random.seed(seed)
    toolbox.register("evaluate", Individual.cma_evaluate, method, error)
    population = toolbox.generate()
    Individual.apply_bounds(population, method, error)

    best_fitness = None
    no_improvement_counter = 0
    best_individual = None
    start_time = time.time()

    try:
        for gen in range(GENERATION_LIMIT):
            if time.time() - start_time > timeout_limit:
                raise TimeoutError(f"Timeout reached for {method} on seed {seed}, error {error}")

            for i, ind in enumerate(population):
                ind.fitness.values = toolbox.evaluate(ind)

            record = stats.compile(population)
            current_best_fitness = min(ind.fitness.values[0] for ind in population)
            hof.update(population)

            if best_fitness is None or current_best_fitness < best_fitness - TOLERANCE:
                best_fitness = current_best_fitness
                no_improvement_counter = 0
                best_individual = hof[0]
            else:
                no_improvement_counter += 1

            if no_improvement_counter >= (NO_IMPROVEMENT_LIMIT + gen / 100):
                strategy.sigma = min(max(strategy.sigma * HARD_SIGMA_INCREASE_FACTOR, 1), 15)
                logging.info(f"Sigma increased: {strategy.sigma}")
                print(f"Sigma increased due to no improvement. New sigma: {strategy.sigma}")
                no_improvement_counter = 0

            toolbox.update(population)
            print(f"Generation {gen}: {record}", end='\r')

            if gen % 500 == 0 and best_individual:
                best_ind = Individual.list_to_ind(best_individual, method, error)
                # best_ind.plot(comparison=False, method=method) # não pode em paralelo

            population = toolbox.generate()
            Individual.apply_bounds(population, method, error)

    except TimeoutError as e:
        logging.warning(e)
        print(e)  # Log timeout event
    except Exception as e:
        logging.error(f"Unexpected error: {e}", exc_info=True)

    finally:
        end_time = time.time()
        execution_time = end_time - start_time

        # Se o melhor indivíduo não existir, salvar valores nulos
        errors_result = best_ind.calc_all_fitness() if best_ind else {
            'ABS_Fitness': None, 'SQUARED_Fitness': None, 'MSE_Fitness': None, 'MABS_Fitness': None
        }

        result_data = {
            'best_ind': best_ind if best_ind else None,
            'error_type': error,
            'method': method,
            'seed': seed,
            'ABS_Fitness': errors_result['ABS_Fitness'],
            'SQUARED_Fitness': errors_result['SQUARED_Fitness'],
            'MSE_Fitness': errors_result['MSE_Fitness'],
            'MABS_Fitness': errors_result['MABS_Fitness'],
            'execution_time': execution_time,
        }

        result_container.append(result_data)
        logging.info(f"Execution finished for {method}, seed {seed}, error {error} in {execution_time:.2f} seconds.")

In [6]:
# Constants
CSV_FILENAME = 'GRN5_CMAES_Parallel.csv'
# Configuração do log
logging.basicConfig(
    filename="GRN5_CMAES_Parallel_Execution.log",
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s"
)

# Check if the CSV exists and load or create it with proper columns
if os.path.exists(CSV_FILENAME):
    results_df = pd.read_csv(CSV_FILENAME)
    existing_results = results_df[['seed', 'error_type', 'method']]
else:
    # Create a new DataFrame with the required columns and write to CSV immediately
    columns = [
        'best_ind', 'error_type', 'method', 'seed', 'ABS_Fitness',
        'SQUARED_Fitness', 'MSE_Fitness', 'MABS_Fitness', 'execution_time'
    ]
    results_df = pd.DataFrame(columns=columns)
    results_df.to_csv(CSV_FILENAME, index=False)  # Create file with headers
    existing_results = pd.DataFrame(columns=['seed', 'error_type', 'method'])

all_methods = ['RK45']
errors = ['ABS', 'SQUARED', 'MSE', 'MABS']

seeds = 3
results = []

TIMEOUT_LIMIT = 18000  # 5h timeout

# Thread-safe queue for results
result_queue = queue.Queue()
write_lock = threading.Lock()

def process_and_save_results():
    """Continuously write results from queue to the CSV file."""
    global results_df
    while True:
        result = result_queue.get()
        if result is None:  # Stop signal
            break
        
        with write_lock:
            results_df = pd.concat([results_df, pd.DataFrame([result])], ignore_index=True)
            results_df.to_csv(CSV_FILENAME, index=False)

def process_seed_error_method(seed, error, method):
    """Run evolution strategy and store results if successful."""
    if not existing_results[
        (existing_results['seed'] == seed) &
        (existing_results['error_type'] == error) &
        (existing_results['method'] == method)
    ].empty:
        logging.info(f"Skipping: seed={seed}, error={error}, method={method}")
        print(f"Skipping already processed: seed={seed}, error={error}, method={method}")
        return

    result_container = []
    thread = threading.Thread(target=run_evolution, args=(seed, error, method, result_container, TIMEOUT_LIMIT))
    thread.start()
    thread.join(timeout=TIMEOUT_LIMIT)

    if thread.is_alive():
        logging.warning(f"Force-stopping {method} for seed {seed}, error {error} due to timeout.")
        print(f"Force-stopping {method} for seed {seed}, error {error} due to timeout.")
        return

    if result_container:
        result_queue.put(result_container[0])  # Send result to queue
    else:
        logging.warning(f"WARNING: No results stored for {method}, seed {seed}, error {error}.")
        print(f"WARNING: No results stored for {method}, seed {seed}, error {error}.")

# Start writer thread
writer_thread = threading.Thread(target=process_and_save_results, daemon=True)
writer_thread.start()

# Parallel execution
with ThreadPoolExecutor() as executor:
    futures = []
    for seed in range(seeds):
        for error in errors:
            for method in all_methods:
                futures.append(executor.submit(process_seed_error_method, seed, error, method))

    # Wait for all tasks to complete
    for future in as_completed(futures):
        future.result()  # Handle exceptions if needed

# Stop writer thread
result_queue.put(None)
writer_thread.join()


Solver: RK45, Error: ABS
Solver: RK45, Error: SQUARED
Solver: RK45, Error: MSE
Solver: RK45, Error: ABS
Solver: RK45, Error: MABS
Solver: RK45, Error: SQUARED
Solver: RK45, Error: MSE
Solver: RK45, Error: MABS
Solver: RK45, Error: ABS
Solver: RK45, Error: SQUARED
Solver: RK45, Error: MSE
Solver: RK45, Error: MABS


C:\Users\anabi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\scipy\integrate\_ivp\common.py:39: UserWarning: The following arguments have no effect for a chosen solver: `min_step`.
  warn("The following arguments have no effect for a chosen solver: {}."


Sigma increased due to no improvement. New sigma: 3.709588964567966335, 'min': 0.38247948884971367, 'max': 0.41646039573088917}5}
Sigma increased due to no improvement. New sigma: 1.45395806547488031895, 'min': 0.3738944170150967, 'max': 0.3766205258742464}8}}ation 84: {'avg': 8.331631703183815, 'std': 0.21401232697001654, 'min': 8.15338569240255, 'max': 8.836658801243095}57}Generation 94: {'avg': 98.47194599935413, 'std': 6.49585308580324, 'min': 85.27189034222755, 'max': 107.06051075451323}Generation 81: {'avg': 0.5576273264726208, 'std': 0.007209665832923148, 'min': 0.5489298685353852, 'max': 0.5710525493876926}
Sigma increased due to no improvement. New sigma: 1.7980701446134241378, 'min': 0.43109630195359466, 'max': 0.486231901150936}152}
Sigma increased due to no improvement. New sigma: 3.056568998003825813, 'min': 90.56217252516707, 'max': 90.61204627378464}2655}3}}eration 276: {'avg': 0.23376630774196513, 'std': 0.0008202381904909654, 'min': 0.23331263206761768, 'max': 0.236467

C:\Users\anabi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\scipy\integrate\_ivp\common.py:39: UserWarning: The following arguments have no effect for a chosen solver: `min_step`.
  warn("The following arguments have no effect for a chosen solver: {}."


Sigma increased due to no improvement. New sigma: 15 0.00012755916601956736, 'min': 0.36731979727014685, 'max': 0.3677516759896283}5}}
Sigma increased due to no improvement. New sigma: 15 0.0001441139050315384, 'min': 0.08760446391144935, 'max': 0.0879248039184817}504}
Sigma increased due to no improvement. New sigma: 2.7957229071849046634e-05, 'min': 0.09845131290280415, 'max': 0.09856485402759499}}ation 4163: {'avg': 2.5892089713048243, 'std': 0.0012023842058925252, 'min': 2.5878270170343187, 'max': 2.5909217781492924}
Sigma increased due to no improvement. New sigma: 10.01254024600123822, 'min': 25.274764433299676, 'max': 25.321344469370768}6583}}}1}
Sigma increased due to no improvement. New sigma: 1.0472354018839503937, 'min': 24.8141527610571, 'max': 24.986963146576155}9034}
Sigma increased due to no improvement. New sigma: 3.21750497718940582, 'min': 2.263777893301129, 'max': 4.123434864620416}8388}6693}}
Sigma increased due to no improvement. New sigma: 1.232217516944518912, 'm

...

C:\Users\anabi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\scipy\integrate\_ivp\common.py:39: UserWarning: The following arguments have no effect for a chosen solver: `min_step`.
  warn("The following arguments have no effect for a chosen solver: {}."


C:\Users\anabi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\scipy\integrate\_ivp\common.py:39: UserWarning: The following arguments have no effect for a chosen solver: `min_step`.
  warn("The following arguments have no effect for a chosen solver: {}."


Sigma increased due to no improvement. New sigma: 1 7.140904272673746e-06, 'min': 0.09556100563911477, 'max': 0.0955818234343656}2}
Sigma increased due to no improvement. New sigma: 1 0.023704804229348626, 'min': 25.259435225522665, 'max': 25.323387931255773}566}}


C:\Users\anabi\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\scipy\integrate\_ivp\common.py:39: UserWarning: The following arguments have no effect for a chosen solver: `min_step`.
  warn("The following arguments have no effect for a chosen solver: {}."


Sigma increased due to no improvement. New sigma: 1.8274227771632159883e-06, 'min': 0.09555348083658557, 'max': 0.09556221834394303}
Sigma increased due to no improvement. New sigma: 1.1513319582251085423e-06, 'min': 0.09554274319348433, 'max': 0.09554976320021247}
Sigma increased due to no improvement. New sigma: 1.424687046713383213e-06, 'min': 0.09554234334449142, 'max': 0.09554827579129192}}
Sigma increased due to no improvement. New sigma: 1.3829800958337914e-05, 'min': 0.09553819388904038, 'max': 0.09580556975323444}}}}
Sigma increased due to no improvement. New sigma: 1: 1.2790075556165808e-07, 'min': 0.09553714733842171, 'max': 0.09553758146010523}
Sigma increased due to no improvement. New sigma: 1 1.7532402959221215e-07, 'min': 0.09553703626309398, 'max': 0.09553761241847739}}
Sigma increased due to no improvement. New sigma: 1: 1.0871557308746508e-07, 'min': 0.0955369983598784, 'max': 0.09553738969407688}}
Sigma increased due to no improvement. New sigma: 1 3.682366904672069

In [7]:
results_df = pd.read_csv(CSV_FILENAME)
results_df

,best_ind,error_type,method,seed,ABS_Fitness,SQUARED_Fitness,MSE_Fitness,MABS_Fitness,execution_time
0,"Individual (fitness=inf, coeffs={'A': {'E': {'...",MSE,RK45,1,91.048484,7.410392,0.219656,0.364194,9804.121740
1,"Individual (fitness=inf, coeffs={'A': {'E': {'...",MSE,RK45,0,62.806442,4.678366,0.087548,0.251226,11347.676752
2,"Individual (fitness=inf, coeffs={'A': {'E': {'...",SQUARED,RK45,2,30.320512,2.587419,0.026779,0.121282,12058.496213
3,"Individual (fitness=inf, coeffs={'A': {'E': {'...",MABS,RK45,1,29.340797,2.772044,0.030737,0.117363,12336.911750
4,"Individual (fitness=inf, coeffs={'A': {'E': {'...",SQUARED,RK45,0,48.030932,3.694710,0.054604,0.192124,12356.517776
5,"Individual (fitness=inf, coeffs={'A': {'E': {'...",MABS,RK45,2,21.742542,1.687286,0.011388,0.086970,12821.676912
6,"Individual (fitness=inf, coeffs={'A': {'E': {'...",ABS,RK45,2,29.385424,2.649536,0.028080,0.117542,12988.416785
7,"Individual (fitness=inf, coeffs={'A': {'E': {'...",SQUARED,RK45,1,28.104718,2.263784,0.020499,0.112419,13314.209897
8,"Individual (fitness=inf, coeffs={'A': {'E': {'...",MSE,RK45,2,40.230081,3.535499,0.049999,0.160920,13708.915587
9,"Individual (fitness=inf, coeffs={'A': {'E': {'...",ABS,RK45,0,24.237360,2.017713,0.016285,0.096949,13750.410479
